<a href="https://colab.research.google.com/github/Kenrich005/ISB_Datathon_23/blob/main/Datathon_ISB_2023_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Dataset_Datathon_Fraud_Detection_28122022.csv',encoding= 'unicode_escape')
df.shape

(10969, 21)

In [ ]:
df.head()

,customer_id,event_dttm_Deli,event_type_nm,client_defined_event_type_nm,event_desc,operaton_amt_100 (rupees),active phone call,remote desktop protokol,emulator,f,...,os,device_model,screensize,agentconnectiontype,screen_hash,device for user,reciever,password change,fast change of ip,ip not equal to previous
0,customer_id_0,2022-10-01 10:51:36,CHANGE_PASSWORD,RENEW,Login password recovery,0.0,NaN,NaN,NaN,St Petersburg,...,NaN,NaN,NaN,NaN,287067538.0,new,old,NaN,NaN,IP differes
1,customer_id_0,2022-10-01 11:15:21,SESSION_SIGNIN,SESSION_SIGNIN,Login,0.0,NaN,NaN,NaN,St Petersburg,...,NaN,NaN,NaN,NaN,287067538.0,new,old,happened on the same day,NaN,IP differes
2,customer_id_0,2022-10-01 11:31:55,SESSION_SIGNIN,SESSION_SIGNIN,Login,0.0,NaN,NaN,NaN,St Petersburg,...,NaN,NaN,NaN,NaN,-205759950.0,new,old,happened on the same day,NaN,IP differes
3,customer_id_0,2022-10-01 11:45:42,SESSION_SIGNIN,SESSION_SIGNIN,Login,0.0,NaN,NaN,NaN,St Petersburg,...,NaN,NaN,NaN,NaN,287067538.0,new,old,happened on the same day,NaN,IP differes
4,customer_id_0,2022-10-03 10:28:24,DEPOSIT,100_BKO,Payment order,116820.0,1.0,False,NaN,NaN,...,iOS,iPhone,375x812,wifi,NaN,NaN,old,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10969 entries, 0 to 10968
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   10969 non-null  object 
 1   event_dttm_Deli               10969 non-null  object 
 2   event_type_nm                 10969 non-null  object 
 3   client_defined_event_type_nm  9554 non-null   object 
 4   event_desc                    10800 non-null  object 
 5   operaton_amt_100 (rupees)     10969 non-null  float64
 6   active phone call             3778 non-null   float64
 7   remote desktop protokol       3786 non-null   object 
 8   emulator                      2923 non-null   float64
 9   f                             4666 non-null   object 
 10  device_id                     3786 non-null   object 
 11  os                            3786 non-null   object 
 12  device_model                  3786 non-null   object 
 13  s

In [ ]:
# Filtering out the events where operation amount is greater than 0
df_amt = df[df['operaton_amt_100 (rupees)']>0]
df_amt.shape

(7481, 21)

In [ ]:
df_amt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7481 entries, 4 to 10968
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   7481 non-null   object 
 1   event_dttm_Deli               7481 non-null   object 
 2   event_type_nm                 7481 non-null   object 
 3   client_defined_event_type_nm  7481 non-null   object 
 4   event_desc                    7481 non-null   object 
 5   operaton_amt_100 (rupees)     7481 non-null   float64
 6   active phone call             1403 non-null   float64
 7   remote desktop protokol       1403 non-null   object 
 8   emulator                      1085 non-null   float64
 9   f                             1755 non-null   object 
 10  device_id                     1403 non-null   object 
 11  os                            1403 non-null   object 
 12  device_model                  1403 non-null   object 
 13  sc

In [ ]:
"""
Required columns for clustering and why
We will intitially chose just few columns for preliminary analysis
All we want to know is if there is any anamoly in the data
The best columns to choose are the ones where the null values are low

customer_id - not required, fraud can happen with anyone, cannot and shouldn't expect any bias there
event_dttm_Deli - required, but more processing needs to be done, to extract day, day_part
event_type_nm - required
client_defined_event_type_nm - for now, not required, event_type_nm gives high level picture
event_desc - for now, not required, event_type_nm gives high level picture
operaton_amt_100 (rupees) - highly important
active phone call - highly important
remote desktop protocol - highly important
emulator - highly important
f - this is city column, Ideally there shouldn't be bias here
    but of all people, Americans are known to be relatively dumber, so might keep it
    for now not required
device_id - not required
os - required, iOS are relatively more robust with respect to security
device_model - not required
screensize - not required
agentconnectiontype - since only two categories, will keep it for now
screen_hash - ideally important, but for now can ignore it
device for user - required
reciever - required
password change - required
fast change of ip - required
ip not equal to previous - required

Summary: required columns
event_type_nm
operaton_amt_100
active phone call
remote desktop protokol
emulator
os
agentconnectiontype
device for user
reciever
password change
fast change of ip
ip not equal to previous

"""
print('')

In [ ]:
df_amt.columns

Index(['customer_id', 'event_dttm_Deli', 'event_type_nm',
       'client_defined_event_type_nm', 'event_desc',
       'operaton_amt_100 (rupees)', 'active phone call',
       'remote desktop protokol', 'emulator', 'f', 'device_id', 'os',
       'device_model', 'screensize', 'agentconnectiontype', 'screen_hash',
       'device for user', 'reciever', 'password change', 'fast change of ip',
       'ip not equal to previous'],
      dtype='object')

In [ ]:
df_filtered = df_amt[['event_type_nm','operaton_amt_100 (rupees)','active phone call',
                 'remote desktop protokol','emulator','os','agentconnectiontype',
                 'device for user','reciever','password change',
                 'fast change of ip','ip not equal to previous']]
df_filtered['active phone call'] = df_filtered['active phone call'].astype('object')
df_filtered['emulator'] = df_filtered['emulator'].astype('object')

df_filtered.shape

<ipython-input-9-1268a34704e9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['active phone call'] = df_filtered['active phone call'].astype('object')
<ipython-input-9-1268a34704e9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['emulator'] = df_filtered['emulator'].astype('object')


(7481, 12)

In [ ]:
df_filtered.isnull().sum()

event_type_nm                   0
operaton_amt_100 (rupees)       0
active phone call            6078
remote desktop protokol      6078
emulator                     6396
os                           6078
agentconnectiontype          7163
device for user              5659
reciever                        0
password change              6937
fast change of ip            7469
ip not equal to previous     6517
dtype: int64

In [ ]:
df_ohe = pd.get_dummies(df_filtered)
df_ohe.shape

(7481, 24)

In [ ]:
df_ohe.columns

Index(['operaton_amt_100 (rupees)', 'event_type_nm_CHECK',
       'event_type_nm_CLIENT_DEFINED', 'event_type_nm_DEPOSIT',
       'event_type_nm_PAYMENT', 'event_type_nm_REQUEST_CREDIT',
       'event_type_nm_WITHDRAW', 'active phone call_0.0',
       'active phone call_1.0', 'remote desktop protokol_False',
       'remote desktop protokol_True', 'emulator_0.0', 'os_Android', 'os_iOS',
       'os_iPadOS', 'agentconnectiontype_cellular', 'agentconnectiontype_wifi',
       'device for user_new', 'device for user_old', 'reciever_new',
       'reciever_old', 'password change_happened on the same day',
       'fast change of ip_happened in 5 minutes',
       'ip not equal to previous_IP differes'],
      dtype='object')

In [ ]:
df_ohe.isnull().sum()

operaton_amt_100 (rupees)                   0
event_type_nm_CHECK                         0
event_type_nm_CLIENT_DEFINED                0
event_type_nm_DEPOSIT                       0
event_type_nm_PAYMENT                       0
event_type_nm_REQUEST_CREDIT                0
event_type_nm_WITHDRAW                      0
active phone call_0.0                       0
active phone call_1.0                       0
remote desktop protokol_False               0
remote desktop protokol_True                0
emulator_0.0                                0
os_Android                                  0
os_iOS                                      0
os_iPadOS                                   0
agentconnectiontype_cellular                0
agentconnectiontype_wifi                    0
device for user_new                         0
device for user_old                         0
reciever_new                                0
reciever_old                                0
password change_happened on the sa

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(2)

In [ ]:
kmeans.fit(df_ohe)
identified_clusters = kmeans.fit_predict(df_ohe)
identified_clusters

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
df_ohe['clusters'] = identified_clusters
df_ohe.clusters.value_counts()

0    7414
1      67
Name: clusters, dtype: int64

In [ ]:
"""
We know that conventionally fraudulent transactions are extremely low
Hence we can safely say that cluster 1 is most likely the fraudulent one
Lets see how the characteristics of those with cluster 1
"""
print('')

In [ ]:
df_ohe[df_ohe['clusters']==1]

,operaton_amt_100 (rupees),event_type_nm_CHECK,event_type_nm_CLIENT_DEFINED,event_type_nm_DEPOSIT,event_type_nm_PAYMENT,event_type_nm_REQUEST_CREDIT,event_type_nm_WITHDRAW,active phone call_0.0,active phone call_1.0,remote desktop protokol_False,...,agentconnectiontype_cellular,agentconnectiontype_wifi,device for user_new,device for user_old,reciever_new,reciever_old,password change_happened on the same day,fast change of ip_happened in 5 minutes,ip not equal to previous_IP differes,clusters
343,3304000.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1022,2126365.9,0,0,0,1,0,0,1,0,1,...,0,0,0,1,1,0,1,0,1,1
1025,2126365.9,0,0,0,1,0,0,1,0,1,...,0,0,0,1,1,0,1,0,1,1
1062,1180000.0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,1,1
1850,1249620.0,0,0,0,0,1,0,1,0,1,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,1956440.0,0,0,0,0,1,0,0,1,1,...,0,0,0,1,0,1,0,0,1,1
7686,1221300.0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,1,1
7687,1221300.0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,1,1
10113,2596000.0,0,0,0,0,1,0,1,0,1,...,1,0,1,0,0,1,0,0,1,1


In [ ]:
df_ohe.groupby('clusters').mean().round(3).T.reset_index().sort_values(by = [1], ascending=False)

clusters,index,0,1
0,operaton_amt_100 (rupees),35285.672,1955626.681
23,ip not equal to previous_IP differes,0.122,0.881
4,event_type_nm_PAYMENT,0.853,0.582
20,reciever_old,0.846,0.552
17,device for user_new,0.122,0.507
9,remote desktop protokol_False,0.185,0.463
19,reciever_new,0.154,0.448
18,device for user_old,0.116,0.448
5,event_type_nm_REQUEST_CREDIT,0.006,0.388
21,password change_happened on the same day,0.070,0.388


In [ ]:
"""
Inferences:
1. Mean Operation amount for cluster 1 is approx 2 mil
2. Most important parameter to differenciate among these two clusters is ip not equal to previous_IP differes
3. iOS has no fraudlent transactions
4. If device for user is new, then high prob of fraud
"""
print('')

In [ ]:
"""
Things to note:
1. Several columns were removed to achieve this result
2. This was only to define which trasactions are most likely fraudulent
3. In one hot encoding, ideally we should drop first to remove multicollinearity, that was not done
4. 
"""
print('')